In [1]:
%matplotlib inline
import mpld3
mpld3.enable_notebook()

from astropy.io import fits
import astropy.units as u
from astroquery import ned
from astropy.wcs import wcs
from astropy.coordinates import SkyCoord
from astropy.table import Table, vstack, hstack
import numpy as np
import tables
from tabulate import tabulate
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from math import ceil

In [2]:
# This is a cell with all self-defined functions
def coord_NED(dwarfname):
    from astroquery.ned.core import RemoteServiceError
    # get rid of the weird symbos *, (, ), etc, since NED doens't like it
    newname = dwarfname.replace('*','').replace('(','').replace(')', '')

    try:
        target = ned.Ned.query_object(newname)
        ned_exist = True
    except RemoteServiceError: 
        print('NED does not have info, skip: %s/%s\n'%(dwarfname, newname))
        ned_exist= False 
    
    if ned_exist == False:
        tRA, tDEC, tVel, tDist = np.nan, np.nan, np.nan, np.nan
    else: # if NED does have this target, then extract coordinates
        tRA = target["RA(deg)"].data.data[0]
        tDEC = target["DEC(deg)"].data.data[0]
        tVel = target["Velocity"].data.data[0]
        tDist = target["Distance (arcmin)"].data.data[0]
    return tRA, tDEC, tDist, tVel, ned_exist

## transform from vhelio to vlsr 
def vhelio2vlsr_Westmeier(vel_init, ral, decb, reverse=False, doradec=True):
    '''
    - from http://www.atnf.csiro.au/people/Tobias.Westmeier/tools_hihelpers.php
    - obj_ra:  should be in degree
    - obj_dec: should be in degree
    - vel_init: velocity that need to be transformed
    -          vel_init = vhelio if reverse = False
    -          vel_init = vlsr   if reverse = True
    - Favor this one than the other one from Rosolowsky
    '''
    from astropy.coordinates import SkyCoord
    import numpy as np

    if doradec==True:
        c = SkyCoord(ral, decb, unit='deg')
        l = np.radians(c.galactic.l.value)
        b = np.radians(c.galactic.b.value)
    else:
        l = np.radians(ral)
        b = np.radians(decb)
    # vlsr 00> vhelio
    if reverse:
        delv = -(9*np.cos(l)*np.cos(b)+12*np.sin(l)*np.cos(b)+7*np.sin(b))
    else:
        delv = +9*np.cos(l)*np.cos(b)+12*np.sin(l)*np.cos(b)+7*np.sin(b)

    # print 'Velocity correction at this (RA, DEC) is (km/s): ', delv
    return vel_init+delv


def find_the_cube(ra, dec, observation='HI4PI'):
    '''
    Use the input (ra, dec) to decide which cube the data locates.

    observation: most of the time, it is HI4PI or GALFA-HI . 
    
    Note: GALFA-HI table is not available now, will do later 
    '''

    clt = Table.read('tables/%s_RADEC.dat'%(observation), format='ascii')
    cramin, cramax = clt['min_ra'], clt['max_ra']
    cdcmin, cdcmax = clt['min_dec'], clt['max_dec']
    indra = np.all([ra>cramin, ra<cramax], axis=0)
    inddc = np.all([dec>cdcmin, dec<cdcmax], axis=0)
    indall = np.where(np.all([indra, inddc], axis=0) == True)

    cubename = clt['cubename'][indall]
    if len(cubename)==0:
        return ''
    else:
        cubename = cubename[0]

        return cubename
    
import sys
import astropy.wcs as wcs
import numpy as np


# a function to read in header and generate RA, DEC and VLSR array
def get_cubeinfo(header, returnHeader=False):
    '''
    A function created to parse the RA, DEC, (and velocity) information from the 2D (3D) header

    - This function has been tested with GALFA-HI/EBHIS cubes, and GALFA-HI 2D images.
    -               also been tested with LAB cubes/images that are in (glon, glat) coordinates.
    - The input header can be 2D: NAXIS=2. NAXIS1 is RA (glon), 2 is DEC (glat)
    -                      or 3D: NAXIS=3. NAXIS1 is RA (glon), 2 is DEC (glat), 3 is Velocity
    - Return: if GALFA-HI or EBHIS:
                        ra and dec in 2D, with shape of (dec.size, ra.size) or (NAXIS2, NAXIS1)
    -                   velocity in 1D.
                        or (ra, dec, vlsr, header array)
    -         if LAB:
                        glon, glat in 2D, with shape of (glat.size, glon.size) or (NAXIS2, NAXIS1)
    -                   velocity in 1D.
                        or (gl, gb, vlsr, header array)
    - History: updated as of 2016.10.03. Yong Zheng @ Columbia Astro.
    '''

    #import sys
    #import astropy.wcs as wcs
    #import numpy as np


    hdrarrs = []
    if header['NAXIS'] == 2:
        hdr2d = header.copy()
        hdrarrs.append(hdr2d)
    elif header['NAXIS'] == 3:
        # create a 2D header (RA/DEC) to speed up the RA/DEC calculation using astropy.wcs
        hdr2d = header.copy()
        # we don't need the velocity (3) information in the header
        delkey = []
        for key in hdr2d.keys():
            if len(key) != 0 and key[-1] == '3': delkey.append(key)
        for i in delkey: del hdr2d[i]

        hdr2d['NAXIS'] = 2
        if 'WCSAXES' in hdr2d.keys(): hdr2d['WCSAXES']=2
        # create a 1D header (vel) to parse the velocity using astropy.wcs
        hdr1d = header.copy()
        # we don't need the RA/DEC keywords info in the header now.
        delkey = []
        for keya in hdr1d.keys():
            if len(keya) != 0 and keya[-1] in ['1', '2']: delkey.append(keya)
        for i in delkey: del hdr1d[i]
        delkey = []
        for keyb in hdr1d.keys():
            if len(keyb) != 0 and keyb[-1] == '3':
                hdr1d.append('%s1'%(keyb[:-1]))
                hdr1d['%s1'%(keyb[:-1])] = hdr1d[keyb]
                delkey.append(keyb)
        for i in delkey: del hdr1d[i]
        hdr1d['NAXIS'] = 1
        if 'WCSAXES' in hdr1d.keys(): hdr1d['WCSAXES']=1

        # save header arrays
        hdrarrs.append(hdr2d)
        hdrarrs.append(hdr1d)
    else:
        print("This code can only handle 2D or 3D data")
        sys.exit(1)

    return_arrays = []

    # calculate RA, DEC
    gwcsa = wcs.WCS(hdr2d)
    n1, n2 = hdr2d['NAXIS1'], hdr2d['NAXIS2']
    ax = np.reshape(np.mgrid[0:n1:1]+1, (1, n1))  # For FITS standard, origin = 1
    ay = np.reshape(np.mgrid[0:n2:1]+1, (n2, 1))  #   then for numpy standard, origin = 0
    coor1, coor2 = gwcsa.all_pix2world(ax, ay, 1) # coor1 = ra  or glon
    return_arrays.append(coor1)                   # coor2 = dec or glat
    return_arrays.append(coor2)

    ## calculate VLSR
    if header['NAXIS'] == 3:
        gwcsb = wcs.WCS(hdr1d)
        n1 = hdr1d['NAXIS1']
        ax = np.mgrid[0:n1:1]+1
        # ax = np.linspace(0, n1, n1)  # nope, wrong
        vel = gwcsb.all_pix2world(ax, 1)[0]
        if 'CUNIT1' in hdr1d.keys():
            if hdr1d['CUNIT1'] in ['m/s', 'M/S', 'M/s', 'm/S']:
                vel = vel/1e3
        else: vel = vel/1e3  # default is usually in m/s
        return_arrays.append(vel)

    if returnHeader == True: return_arrays.append(hdrarrs)
    return return_arrays

In [3]:
def ellipse_mask_cube(semi_a, semi_b, PA, tar_x, tar_y, cube_header, extend_pix = 0):
    ## make ellipse based on the semi major/minor axies and position angle of the targets 
    
    cube_wh = cube_header['NAXIS1']  # RA
    cube_ht = cube_header['NAXIS2']  # DEC
    x1d = np.arange(cube_wh)
    y1d = np.arange(cube_ht)
    x2d, y2d = np.meshgrid(x1d, y1d)

    # center at the source 
    x2d_nc, y2d_nc = x2d-tar_x, y2d-tar_y

    # this is a rough pixel length for the semimajor and semiminor axis
    semi_a_pix = ceil(abs(semi_a / cube_header['CDELT1']))
    semi_b_pix = ceil(abs(semi_b / cube_header['CDELT1']))
    
    #phi = np.radians(90-PA) changing 
    phi = np.radians(90+PA)

    # transform to new coordinate system
    xx_prime = x2d_nc*np.cos(phi) + y2d_nc*np.sin(phi)
    yy_prime = -x2d_nc*np.sin(phi) + y2d_nc*np.cos(phi)

    mask_ep = ((xx_prime/(semi_a_pix+extend_pix))**2 + (yy_prime/(semi_b_pix+extend_pix))**2)<=1 
    return mask_ep

In [12]:
def find_info(objname):
    from astropy.table import Table 
    tb = Table.read('/Users/amalyajohnson/Desktop/astro/Dwarf Galaxies/dwarfgalcomplete.txt', format='ascii')
    
    newname = objname.replace('*','').replace('(','').replace(')', '').replace(' ', '')
    for i, iname in enumerate(tb['GalaxyName']):
        i_tbnewname = iname.replace('*','').replace('(','').replace(')', '').replace(' ', '')
        
        if newname == i_tbnewname:
            find_it = True
            break
        else: 
            find_it = False
            
    if find_it == True:
        print('YES, we find it!!!', tb['RA'][i], tb['DEC'][i], tb['vh(m/s)'][i])
        return tb['RA'][i], tb['DEC'][i], tb['vh(m/s)'][i], tb['rh(arcmins)'][i], tb['e=1-b/a'][i], tb['PA'][i]
    else:
        print('Sorry, not there, check your object name. ')
        return np.nan, np.nan, np.nan
    
    
# Galaxy  info

objname = 'WLM'
#objRA, objDEC objV = 132.874, 63.13, -116000
#semi_a = .073 # degree, semi major axes
#semi_b = .03796 # degree, semi minor axes
#PA = 85      # degree, I suppose it's angle between semi_a and north, measured toward east? 

### 
objRA, objDEC, objV_helio, a, ecc, PA = find_info(objname)
vcoord = vhelio2vlsr_Westmeier(0, objRA, objDEC, doradec=True) # in km/s
objV = objV_helio + vcoord*1e3
semi_b = (1 - ecc)*a/60
semi_a = a/60 



YES, we find it!!! 0.4917 -15.4608 -130000


In [13]:
cubename = '/Users/amalyajohnson/Desktop/datacubes/hi4pi need//CAR_D01.fits'
img1 = fits.getdata(cubename)
hdr = fits.getheader(cubename)

In [14]:
# setting up the WCS
wcsleop =  wcs.WCS(hdr)
 

# converting to pixel space
tar_x, tar_y, tar_v = wcsleop.all_world2pix(objRA, objDEC, objV, 0)
tar_x = int(tar_x)
tar_y = int(tar_y)
tar_v = int(tar_v)
# & checking it's in there
img1.shape,  tar_v, tar_x, tar_y


((933, 266, 266), 362, 247, 186)

In [15]:
## figure out how large the area surrounding the source that we want 
ep1 = ellipse_mask_cube(semi_a, semi_b, PA, tar_x, tar_y, hdr, extend_pix=0)
ep2 = ellipse_mask_cube(.25, .25, PA, tar_x, tar_y, hdr, extend_pix=0)

#ep3 if need to go bigger/smaller (resolved detected)
ep3 = ellipse_mask_cube(semi_a, semi_b, PA, tar_x, tar_y, hdr, extend_pix=2)

#ep3 beamsize (unresolved detected)
#ep3 = ellipse_mask_cube(0.133333, 0.133333, PA, tar_x, tar_y, hdr, extend_pix=0) #hi4pi
#ep3 = ellipse_mask_cube(0.0333333, 0.0333333, PA, tar_x, tar_y, hdr, extend_pix=0) #galfa


a = img1[tar_v][ep1]
b = img1[tar_v][ep2]
semi_a, semi_b

(0.14366666666666666, 0.064649999999999985)

In [18]:
def vel_width(objname):
    tb = Table.read('/Users/amalyajohnson/Desktop/astro/Dwarf Galaxies/dwarfgalcomplete_newnew.txt', format='ascii')    
    
    for i, iname in enumerate(tb['GalaxyName']):        
        if iname == objname:
            find_it = True
            break
        else: 
            find_it = False
            
    if find_it == True:
        sig = float(tb['sigma_g(km/s)'][i])
        fwhm = sig*2.355
        #hi4pi = 1.29
        galfa = .736
        v_width = fwhm/galfa #or galfa
        print('found', objname, sig, fwhm, v_width)
        return ceil(v_width) # x would be an int 
    else:
        print('Sorry, not there, check your object name. ')
        return np.nan, np.nan, np.nan
        
x = 60 #vel_width(objname)

## if x is odd number,  add 1 to it 
if np.mod(x, 2) == 1: x=x+1 

full_chan = tar_v + np.arange(x+1) - ceil(x/2)
vrange = []
vrange_std = []
for iv in full_chan:
    v_chan  = img1[iv][ep2] #changed to 2 for a WLM 
    v_std = np.nanstd(v_chan)
    vrange.append(v_chan)
    vrange_std.append(v_std)
    
vrange = np.asarray(vrange)
vrange_std = np.asarray(vrange_std)
vrange_sum = np.nansum(vrange)

#print(vrange_std, vrange)
vrange_sum

1349.1046

In [ ]:
x

In [ ]:
plt.figure(figsize=(8, 8))
im = plt.imshow(img1[tar_v, :, :], cmap='jet', origin='lower') #, vmin=-.4, vmax=.4)
plt.contour(ep1, colors='white')
plt.contour(ep2, colors='r') #, alpha=0.1)
plt.contour(ep3, colors='white') #, alpha=0.1)

plt.text(tar_x+15, tar_y+15, objname, color='w', fontsize=14, weight='semibold')
plt.title(objname)
plt.colorbar(im)

#plt.xlim(200, 250)
#plt.ylim(125, 175)

#plt.savefig('/Users/amalyajohnson/Desktop/images/%s_image2.pdf'%(objname))
#plt.savefig('/Users/amalyajohnson/Desktop/test.pdf')

plt.show()


In [ ]:
dist = 769.13/1e3
tflux = 433.7005
tmasssolar = tflux*hdr['CDELT3']/1000.*1.8e18*np.cos(objDEC*np.pi/180.)*(hdr['CDELT2']*np.pi/180.*dist*u.pc.in_units('cm')*1e6)**2*1.6727e-24/1.99e33
tmasssolar/1e6

In [ ]:

#a velocity channels
a_1 = (img1[tar_v-5][ep1])
a_2 = (img1[tar_v-4][ep1])
a_3 = (img1[tar_v-3][ep1])
a_4 = (img1[tar_v-2][ep1])
a_5 = (img1[tar_v-1][ep1])
a_6 = (img1[tar_v][ep1])
a_7 = (img1[tar_v+1][ep1])
a_8 = (img1[tar_v+2][ep1])
a_9 = (img1[tar_v+3][ep1])
a_10 = (img1[tar_v+4][ep1])
a_11 = (img1[tar_v+5][ep1])
arange = np.array([a_1, a_2, a_3, a_4, a_5, a_6, a_7, a_8, a_9, a_10, a_11])
arange_std = np.array([np.nanstd(a_1), np.nanstd(a_2), np.nanstd(a_3), np.nanstd(a_4),
                       np.nanstd(a_5), np.nanstd(a_6), np.nanstd(a_7), np.nanstd(a_8),
                       np.nanstd(a_9), np.nanstd(a_10), np.nanstd(a_11)])


#b velocity channels     
b_1 = (img1[tar_v-5][ep2])
b_2 = (img1[tar_v-4][ep2])
b_3 = (img1[tar_v-3][ep2])
b_4 = (img1[tar_v-2][ep2])
b_5 = (img1[tar_v-1][ep2])
b_6 = (img1[tar_v][ep2])
b_7 = (img1[tar_v+1][ep2])
b_8 = (img1[tar_v+2][ep2])
b_9 = (img1[tar_v+3][ep2])
b_10 = (img1[tar_v+4][ep2])
b_11 = (img1[tar_v+5][ep2])
brange = np.array([b_1, b_2, b_3, b_4, b_5, b_6, b_7, b_8, b_9, b_10, b_11])
brange_std = np.array([np.nanstd(b_1), np.nanstd(b_2), np.nanstd(b_3), np.nanstd(b_4), 
                       np.nanstd(b_5), np.nanstd(b_6), np.nanstd(b_7), np.nanstd(b_8), 
                       np.nanstd(b_9), np.nanstd(b_10), np.nanstd(b_11)])

std_a = np.nanstd(a)
med_stda = np.nanmedian(arange_std)
std_b = np.nanstd(b)
med_stdb = np.nanmedian(brange_std)
tflux_a = np.nansum(arange) #flux throughout  entire velocity range of +/- 5 channels
med_tfluxa = np.nanmedian(arange)
tflux_b = np.nansum(brange)
med_tfluxb = np.nanmedian(brange)
flux_a = np.nansum(a)
flux_b = np.nansum(b)

print(med_stdb)


In [ ]:
#UNRESOLVED DETECTED

#hi4pi
ep3 = ellipse_mask_cube(0.133333, 0.133333, PA, tar_x, tar_y, hdr, extend_pix=0)

#galfa
#ep3 = ellipse_mask_cube(0.0333333, 0.0333333, PA, tar_x, tar_y, hdr, extend_pix=0)

c = img1[tar_v][ep3]

#c velocity channels
c_1 = (img1[tar_v-5][ep3])
c_2 = (img1[tar_v-4][ep3])
c_3 = (img1[tar_v-3][ep3])
c_4 = (img1[tar_v-2][ep3])
c_5 = (img1[tar_v-1][ep3])
c_6 = (img1[tar_v][ep3])
c_7 = (img1[tar_v+1][ep3])
c_8 = (img1[tar_v+2][ep3])
c_9 = (img1[tar_v+3][ep3])
c_10 = (img1[tar_v+4][ep3])
c_11 = (img1[tar_v+5][ep3])
                                   
crange = np.array([c_1, c_2, c_3, c_4, c_5, c_6, c_7, c_8, c_9, c_10, c_11])
crange_std = np.array([np.nanstd(c_1), np.nanstd(c_2), np.nanstd(c_3), np.nanstd(c_4),
                       np.nanstd(c_5), np.nanstd(c_6), np.nanstd(c_7), np.nanstd(c_8),
                       np.nanstd(c_9), np.nanstd(c_10), np.nanstd(c_11)])

std_c = np.nanstd(c)
med_stdc = np.nanmedian(crange_std)
tflux_c = np.nansum(crange) #flux throughout  entire velocity range of +/- 5 channels
med_tfluxc = np.nanmedian(crange)
flux_c = np.nansum(c)

#flux_c, tflux_c, flux_a, tflux_a
c.size

In [ ]:
plt.figure(figsize=(8, 8))
im = plt.imshow(img1[tar_v, :, :], cmap='jet', origin='lower', vmin=-.5, vmax=.5)
plt.contour(ep1, colors='white')
plt.contour(ep2, colors='r') #, alpha=0.1)
plt.text(tar_x+15, tar_y+15, objname, color='w', fontsize=14, weight='semibold')
plt.title(objname)
plt.colorbar(im)

plt.contour(ep3, colors='k', alpha=.5) #for unresolved detected

#plt.xlim(200, 265)
#plt.ylim(125, 200)

#plt.savefig('/Users/amalyajohnson/Desktop/images/%s_image2.pdf'%(objname))
#plt.savefig('/Users/amalyajohnson/Desktop/test.pdf')

plt.show()


# Tables (from other notebook)

In [ ]:
#GALDATA NEW
#tbnew = Table(names=('GalaxyName', 'std', 'std_med', 'std30', 'std30_med', 'flux', 'tflux', 'tflux_med', 'flux30', 'tflux30', 'tflux30_med'), meta={'name': 'first table'},dtype=('<U11', 'f8', 'f8', 'f8', 'f8', 'f8', 'f8', 'f8', 'f8', 'f8', 'f8'))
#tbnew = Table.read('/Users/amalyajohnson/Desktop/astro/Dwarf Galaxies/galdata_new.txt', format='ascii')
#row = np.array([objname, std_a, med_stda, std_b, med_stdb, flux_a, tflux_a, med_tfluxa, flux_b, tflux_b, med_tfluxb])
#tbnew.add_row(row)

#tbnew
#f = open('galdata_new.txt', 'w')
#table = tbnew
#f.write(tabulate(tbnew))
#f.close()


In [ ]:
#tbv = Table.read('/Users/amalyajohnson/Desktop/astro/Dwarf Galaxies/stds.txt', format='ascii')


In [ ]:
rowv = np.array([objname, np.nanstd(a_1), np.nanstd(a_2), np.nanstd(a_3), np.nanstd(a_4),
                       np.nanstd(a_5), np.nanstd(a_6), np.nanstd(a_7), np.nanstd(a_8),
                       np.nanstd(a_9), np.nanstd(a_10), np.nanstd(a_11)])
rowv_ex = np.array(['%s_ex'%(objname), np.nanstd(b_1), np.nanstd(b_2), np.nanstd(b_3), np.nanstd(b_4), 
                       np.nanstd(b_5), np.nanstd(b_6), np.nanstd(b_7), np.nanstd(b_8), 
                      np.nanstd(b_9), np.nanstd(b_10), np.nanstd(b_11)])

In [ ]:
tbv.add_row(rowv)
tbv.add_row(rowv_ex)

In [ ]:
f = open('stds.txt', 'w')
table = tbv
f.write(tabulate(tbv))
f.close()